In [1]:
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import re
import gzip

In [2]:
def parseData(fname):
    for l in gzip.open(fname):
        yield eval(l)

data = list(parseData("C:/Users/iisab/OneDrive/Documents/School/Fall2023/CSE158/Assignment2/australian_user_reviews.json.gz"))

dm = [[0,0],[0,0]]

users = set()
games = set()

nodate = 0

reviews = []

for user in data:
    if user["user_id"] in users:
        #print(f"ducplicate user skipped: {user['user_id']}")
        pass
    else:
        users.add(user["user_id"])
        for review in user["reviews"]:
            games.add(review["item_id"])
            funny = review["funny"]
            hasfunny = int(funny != "")
            if funny == "":
                review["funny"] = 0
            else:
                review["funny"] = int(re.findall("\d+", funny)[0])
                
            helpful = review["helpful"]
            hashelpful = int(helpful != "No ratings yet")
            if helpful == "No ratings yet":
                review["helpful_n"] = 0
                review["helpful_total"] = 0
                review["helpful"] = 0
            else:
                nums = re.findall("\d+", helpful.replace(",", ""))
                helpfulness = float(nums[0]) / float(nums[1])
                review["helpful"] = float(nums[0]) / float(nums[1])
                review["helpful_n"] = float(nums[0])
                review["helpful_total"] = float(nums[1])
            
            dm[hasfunny][hashelpful] += 1

            try:
                post_datetime = datetime.strptime(review["posted"],'Posted %B %d, %Y.')
                review["posted"] = post_datetime
            except:
                nodate += 1

            review["user_id"] = user["user_id"]
            review["user_url"] = user["user_url"]
            reviews.append(review)

In [3]:
from collections import defaultdict
import string
from nltk.stem.porter import *

wordCount = defaultdict(int)
punctuation = set(string.punctuation)
stemmer = PorterStemmer()
for review in reviews:
	r = ''.join([c for c in review['review'].lower() if not c in punctuation])
	for w in r.split():
		w = stemmer.stem(w)
		wordCount[w] += 1
		
len(wordCount)

97248

In [8]:
counts = [(wordCount[w], w) for w in wordCount]
counts.sort()
counts.reverse()
words = [x[1] for x in counts[:1000]]
wordId = dict(zip(words, range(len(words))))
wordSet = set(words)
words

['the',
 'game',
 'and',
 'a',
 'to',
 'it',
 'is',
 'i',
 'of',
 'you',
 'thi',
 'in',
 'for',
 'that',
 'play',
 'with',
 'but',
 'have',
 'on',
 'be',
 'are',
 'get',
 'as',
 'your',
 'like',
 'if',
 'good',
 'can',
 'not',
 'fun',
 'my',
 'so',
 'great',
 'all',
 'just',
 'wa',
 'or',
 'one',
 'veri',
 'there',
 'more',
 'me',
 'at',
 'do',
 'realli',
 'will',
 'time',
 'would',
 'an',
 'ha',
 'make',
 'no',
 'best',
 'from',
 'out',
 'some',
 'up',
 'when',
 'dont',
 'they',
 '3',
 'buy',
 'what',
 'love',
 'onli',
 'even',
 'much',
 'by',
 'go',
 'then',
 'which',
 'other',
 'about',
 'peopl',
 'want',
 'de',
 'still',
 'amaz',
 '2',
 'recommend',
 '♥♥♥♥',
 'thing',
 'ever',
 'awesom',
 'stori',
 'hour',
 'hat',
 'into',
 'well',
 'look',
 'becaus',
 'friend',
 'than',
 'new',
 'also',
 'use',
 'first',
 '1010',
 'player',
 'now',
 'need',
 'que',
 'who',
 'how',
 'better',
 'gameplay',
 'most',
 'enjoy',
 'feel',
 'say',
 'kill',
 'had',
 'lot',
 'mani',
 'bad',
 'worth',
 'way'

In [ ]:
# NOT GONNA USE DIDN'T PERFORM WELL AT ALL
# wordCount = defaultdict(int)
# punctuation = set(string.punctuation)
# def feature(datum):
#     feat = [0]*len(words)
#     r = ''.join([c for c in datum['review'].lower() if not c in punctuation])
#     ws = r.split()
#     ws2 = [' '.join(x) for x in list(zip(ws[:-1],ws[1:]))]
#     ws3 = [' '.join(x) for x in list(zip(ws[:-2],ws[1:-1],ws[2:]))]
#     ws4 = [' '.join(x) for x in list(zip(ws[:-3],ws[1:-2],ws[2:-1],ws[3:]))]
#     ws5 = [' '.join(x) for x in list(zip(ws[:-4],ws[1:-3],ws[2:-2],ws[3:-1],ws[4:]))]
#     for w in ws + ws2 + ws3 + ws4 + ws5:
#         if w in words:
#             feat[wordId[w]] += 1
#     feat.append(1) #offset
#     return feat

In [5]:
from sklearn.model_selection import train_test_split
import numpy as np

# X_train, X_test, Y_funny_train, Y_funny_test, Y_helpful_train, Y_helpful_test = [], [], [], [], [], []

# train, test = train_test_split(reviews, test_size=0.25, random_state=0)

# for review in train:
#     X_train.append(feature(review))
#     #Y1.append(review["funny"])
#     if review["funny"] == 0:
#         Y_funny_train.append(0)
#     else:
#         Y_funny_train.append(np.log(review["funny"]))
#     #Y2.append(review["helpful_n"])
#     if review["helpful_n"] == 0:
#         Y_helpful_train.append(0)
#     else:
#         Y_helpful_train.append(np.log(review["helpful_n"]))

# for review in test:
#     X_test.append(feature(review))
#     #Y1.append(review["funny"])
#     if review["funny"] == 0:
#         Y_funny_test.append(0)
#     else:
#         Y_funny_test.append(np.log(review["funny"]))
#     #Y2.append(review["helpful_n"])
#     if review["helpful_n"] == 0:
#         Y_helpful_test.append(0)
#     else:
#         Y_helpful_test.append(np.log(review["helpful_n"]))

In [ ]:
# from sklearn.metrics import mean_squared_error, mean_absolute_error
# guess_mean_funny = np.mean(Y_funny_train)
# guess_mean_helpful = np.mean(Y_helpful_train)

# print("baseline", mean_squared_error(Y_funny_test, [guess_mean_funny]*len(Y_funny_test)), mean_squared_error(Y_helpful_test, [guess_mean_helpful]*len(Y_helpful_test)))

baseline 0.17818192454918605 0.557911382661004


In [23]:
from sklearn import linear_model

# Cs = [0.01, 0.1, 1, 10, 100, 1000, 10000]

# for C in Cs:

#     model1 = linear_model.Ridge(C, fit_intercept=True)
#     model1.fit(X_train, Y_funny_train)

#     model2 = linear_model.Ridge(C, fit_intercept=True)
#     model2.fit(X_train, Y_helpful_train)

#     pred_funny_test = model1.predict(X_test)
#     pred_helpful_test = model2.predict(X_test)

#     print(C, mean_squared_error(Y_funny_test, pred_funny_test), mean_squared_error(Y_helpful_test, pred_helpful_test))

In [ ]:
# Cs = [0.01, 0.025, 0.05, 0.1, 0.25, 0.5, 1]
# for C in Cs:

#     model1 = linear_model.Lasso(C, fit_intercept=True)
#     model1.fit(X_train, Y_funny_train)

#     model2 = linear_model.Lasso(C, fit_intercept=True)
#     model2.fit(X_train, Y_helpful_train)

#     pred_funny_test = model1.predict(X_test)
#     pred_helpful_test = model2.predict(X_test)

#     print(C, mean_squared_error(Y_funny_test, pred_funny_test), mean_squared_error(Y_helpful_test, pred_helpful_test))

0.01 0.17730062713547182 0.5386878415592938
0.025 0.1778278432578511 0.539897773766243
0.05 0.1779909555463131 0.5411473583167075
0.1 0.17818192454918605 0.543156420319067
0.25 0.17818192454918605 0.5475217247375852
0.5 0.17818192454918605 0.557911382661004
1 0.17818192454918605 0.557911382661004


In [6]:
words[0]

'the'

In [18]:
import math
df = defaultdict(int)

train, test = train_test_split(reviews, test_size=0.25, random_state=0)

for d in train:
    r = ''.join([c for c in d['review'].lower() if not c in punctuation])
    # for each word in the text
    for w in set(r.split()):
        # if this particular word is one we are looking for add 1 to document frequency
        if w in words:
            df[w] += 1

print([df[w] for w in words])

[16991, 22303, 15967, 16597, 14230, 13290, 14216, 13021, 12547, 10554, 3, 8458, 8800, 6994, 5774, 7940, 8139, 6378, 6107, 4973, 5099, 5120, 4455, 4356, 5462, 5517, 6249, 4769, 5061, 6170, 4932, 5311, 5323, 4508, 4504, 5, 3682, 4138, 6, 3110, 3644, 3702, 3568, 3157, 0, 3288, 3081, 3995, 3517, 46, 2049, 3112, 3894, 2952, 3247, 2864, 2823, 2920, 3064, 2393, 1408, 2383, 2580, 2426, 0, 2641, 2627, 2375, 1687, 2381, 1923, 2040, 2294, 0, 1838, 1076, 2379, 0, 2226, 2271, 1783, 1058, 2543, 3, 0, 323, 136, 2009, 2094, 813, 2, 429, 1985, 1706, 1853, 901, 1906, 2335, 839, 1887, 1300, 833, 1833, 1800, 1900, 1892, 1817, 791, 1079, 1413, 868, 1702, 1161, 2, 1237, 1851, 1459, 1813, 1428, 1265, 1450, 0, 100, 1697, 938, 1529, 1614, 952, 0, 939, 1471, 1361, 1446, 1445, 1, 3, 1205, 801, 718, 879, 1424, 1229, 1310, 1344, 741, 1353, 12, 1402, 375, 1215, 1285, 11, 1204, 1213, 722, 1304, 0, 1214, 651, 1228, 728, 506, 1153, 1170, 1184, 1090, 966, 512, 997, 479, 3, 1135, 668, 1084, 726, 103, 1059, 4, 0, 414, 10

In [19]:
def feature(datum):
    feat = [0]*len(words)
    r = ''.join([c for c in datum['review'].lower() if not c in punctuation])
    for word in r.split():
        if word in words:
            feat[wordId[word]] += 1
    # offset
    
    return [1] + [feat[wordId[w]] * math.log10(len(train) / df[w]) for w in words if df[w] > 0]

In [57]:
baseline = len([0 for review in reviews if review["helpful_n"] < 1])/len(reviews)

22699

In [64]:
# helpful ratings threshold: calculating how many reviews were rated helpful by a certain number of people.
threshold = 100
num = np.sum([0 if review["helpful_n"] < threshold else 1 for review in reviews])
num/len(reviews)

0.004295738490501455

In [47]:
threshold = 1

np.sum([0 if review["helpful_n"] < threshold else 1 for review in reviews]) / len(reviews)
1 - np.sum([0 if review["helpful_n"] < threshold else 1 for review in reviews]) / len(reviews)

0.6115180557932569

In [36]:
len(reviews)

58430

In [20]:
# baseline len(neg set)/total len about 58000 randomly guessing
baseline_train = len([0 for review in train if review["helpful_n"] < 1])/len(train)
baseline_train

0.6110401168362922

In [35]:
len(train)

43822

In [33]:
baseline_test = len([0 for review in test if review["helpful_n"] < 1])/len(test)
baseline_test

0.6129518072289156

In [34]:
len(test)

14608

In [48]:
X_train = [feature(review) for review in train]
Y_funny_train = [1 if review["funny"] > 25 else 0 for review in train]
Y_helpful_train = [1 if review["helpful_n"] > 1 else 0 for review in train] # try with higher threshold 5, 10

X_test = [feature(review) for review in test]
Y_funny_test = [1 if review["funny"] > 25 else 0 for review in test]
Y_helpful_test = [1 if review["helpful_n"] > 1 else 0 for review in test] # try with higher threshold 5, 10


In [28]:
#mod_funny = linear_model.LogisticRegression(C=1, max_iter=1000)
mod_helpful = linear_model.LogisticRegression(C=1, max_iter=1000)

In [49]:
#mod_funny.fit(X_train, Y_funny_train)
mod_helpful.fit(X_train, Y_helpful_train)

#predictions_funny = mod_funny.predict(X_test)
predictions_helpful = mod_helpful.predict(X_test)

#correct_funny = predictions_funny == Y_funny_test
correct_helpful = predictions_helpful == Y_helpful_test

In [50]:
#accuracy_funny = sum(correct_funny) / len(correct_funny)
accuracy_helpful = sum(correct_helpful) / len(correct_helpful)
#print("accuracy_funny: " + str(accuracy_funny) + ", accuracy_helpful: " + str(accuracy_helpful))
print("accuracy_helpful: " + str(accuracy_helpful))

accuracy_helpful: 0.7983296823658269
